In [4]:
from utils.config import *
import pandas as pd
import numpy as np
from utils import dataset as ds
from utils import emg_handler
import efcp_routine
import os
# reload modules:
import importlib

tmp = ds.movload(os.path.join(DATA_PATH, 'behavioural', 'subj100', 'efcp_100_01.mov'))

Trial:  1
Trial:  2
Trial:  3
Trial:  4
Trial:  5
Trial:  6
Trial:  7
Trial:  8
Trial:  9
Trial:  10
Trial:  11
Trial:  12
Trial:  13
Trial:  14
Trial:  15
Trial:  16
Trial:  17
Trial:  18
Trial:  19
Trial:  20
Trial:  21
Trial:  22
Trial:  23
Trial:  24
Trial:  25
Trial:  26
Trial:  27
Trial:  28
Trial:  29
Trial:  30


In [6]:
importlib.reload(efcp_routine)

subject = 100
efcp_routine.subject_routine(subject=subject, fs_force=500, fs_emg=2148.1481, bpf=[20, 500], lpf=30, debug=0)

Processing block 1
Trial:  1
Trial:  2
Trial:  3
Trial:  4
Trial:  5
Trial:  6
Trial:  7
Trial:  8
Trial:  9
Trial:  10
Trial:  11
Trial:  12
Trial:  13
Trial:  14
Trial:  15
Trial:  16
Trial:  17
Trial:  18
Trial:  19
Trial:  20
Trial:  21
Trial:  22
Trial:  23
Trial:  24
Trial:  25
Trial:  26
Trial:  27
Trial:  28
Trial:  29
Trial:  30


/Users/alighavampour/Desktop/Projects/EFC_patternfMRI/efcp_routine.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_mov = pd.concat([df_mov, tmp], ignore_index=True)


Processing block 2
Trial:  1
Trial:  2
Trial:  3
Trial:  4
Trial:  5
Trial:  6
Trial:  7
Trial:  8
Trial:  9
Trial:  10
Trial:  11
Trial:  12
Trial:  13
Trial:  14
Trial:  15
Trial:  16
Trial:  17
Trial:  18
Trial:  19
Trial:  20
Trial:  21
Trial:  22
Trial:  23
Trial:  24
Trial:  25
Trial:  26
Trial:  27
Trial:  28
Trial:  29
Trial:  30
Processing block 3
Trial:  1
Trial:  2
Trial:  3
Trial:  4
Trial:  5
Trial:  6
Trial:  7
Trial:  8
Trial:  9
Trial:  10
Trial:  11
Trial:  12
Trial:  13
Trial:  14
Trial:  15
Trial:  16
Trial:  17
Trial:  18
Trial:  19
Trial:  20
Trial:  21
Trial:  22
Trial:  23
Trial:  24
Trial:  25
Trial:  26
Trial:  27
Trial:  28
Trial:  29
Trial:  30
Processing block 4
Trial:  1
Trial:  2
Trial:  3
Trial:  4
Trial:  5
Trial:  6
Trial:  7
Trial:  8
Trial:  9
Trial:  10
Trial:  11
Trial:  12
Trial:  13
Trial:  14
Trial:  15
Trial:  16
Trial:  17
Trial:  18
Trial:  19
Trial:  20
Trial:  21
Trial:  22
Trial:  23
Trial:  24
Trial:  25
Trial:  26
Trial:  27
Trial:  28
Tr

/Users/alighavampour/Desktop/Projects/EFC_patternfMRI/efcp_routine.py:74: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  file = pd.read_csv(fname, header=None, delimiter=',', skiprows=5)


TypeError: slice indices must be integers or None or have an __index__ method